# 0. Import important library

In [19]:
import numpy as np
import pandas as pd
from tkinter import *
from tkinter.filedialog import askopenfilename
import re
from decimal import Decimal

# 1. Import G-code from the user (from google drive link)

In [2]:
import gdown
import os
from IPython.display import display
from ipywidgets import Text, Button, VBox

In [3]:
def process_dataset():
    link_input = Text(
        value='',
        placeholder='Paste your Google Drive share link here',
        description='Drive Link:',
        style={'description_width': 'initial'},
        layout={'width': '50%'}
    )

    def on_click(b):
        try:
            print("Downloading dataset...")
            url = link_input.value

            # Convert share URL to direct download URL if needed
            if 'drive.google.com/file/d/' in url:
                file_id = url.split('/file/d/')[1].split('/')[0]
                url = f'https://drive.google.com/uc?id={file_id}'

            output = '/content/data.gcode'
            gdown.download(url, output, fuzzy=True)
            print("Download complete!")

        except Exception as e:
            print(f"Error: {str(e)}")

    process_button = Button(description='Process gcode', button_style='primary')
    process_button.on_click(on_click)
    display(VBox([link_input, process_button]))

# Install gdown if not already installed
!pip install -q gdown --upgrade

# Execute
process_dataset()

Downloading...
From: https://drive.google.com/uc?id=1a9gaFnPHH1SVmhjdsBfgU5Wvx_TWzeAC
To: /content/data.gcode
100%|██████████| 366/366 [00:00<00:00, 762kB/s]

Download complete!


In [5]:
file = '/content/data.gcode'
print(file)

/content/data.gcode


# 2. Extract information from the G-code



In [6]:
def extract_filament_used(filename):
    with open(filename, 'r') as file:
        for line in file:
            if line.startswith("; total filament used [g] = "):
                return float(line.split("=")[1].strip())

def extract_printing_time_seconds(filename):
    with open(filename, 'r') as file:
        for line in file:
            if line.startswith("; estimated printing time (normal mode) = "):
                time_str = line.split("=")[1].strip()

                # Extract hours, minutes, and seconds
                match = re.match(r"(\d+)h (\d+)m (\d+)s", time_str)
                if match:
                    hours, minutes, seconds = map(int, match.groups())
                    return hours * 3600 + minutes * 60 + seconds

                match = re.match(r"(\d+)m (\d+)s", time_str)
                if match:
                    minutes, seconds = map(int, match.groups())
                    return minutes * 60 + seconds

                match = re.match(r"(\d+)s", time_str)
                if match:
                    return int(match.group(1))
    return None

In [7]:
Filament_used = extract_filament_used(file)
Time_used = extract_printing_time_seconds(file)

print("Filament Used:",Filament_used,"g")
print("Time Used:",Time_used,"Seconds")

Filament Used: 24.36 g
Time Used: 4360 Seconds


In [13]:
"""Config Variable"""
Price_per_Kg = 469
Time_cost = 12
Time_cost_expo = 0.2

"""Utility cost"""
Price_per_unit = 7
Power_usage = 600

"""Maintenance cost 1 to 0"""
Maintenance = 0.6

In [18]:
Plastic_cost = round(Filament_used * (Price_per_Kg * 0.001),2)
Time_cost_ovl = round(pow((Time_used / 60),Time_cost_expo) * Time_cost,2)
Electricity_Cost = round((Time_used / 3600) * (Power_usage / 1000) * Price_per_unit,2)
Maintenance_Cost = round((Plastic_cost + Time_cost) * Maintenance,2)

Total_Price = round(Plastic_cost + Time_cost + Electricity_Cost + Maintenance_Cost,2)

print("Plastic cost:",Plastic_cost)
print("Time cost:",Time_cost_ovl)
print("Electricity cost:",Electricity_Cost)
print("Maintenance cost:",Maintenance_Cost)

print("Total = ",Total_Price, "baht")

Plastic cost: 11.42
Time cost: 66.64
Electricity cost: 5.09
Maintenance cost: 23.82
Total =  68.61 baht
